# Read file

In [1]:
%%writefile romy.txt
First line is first and second is also first.
Second line is third and third is given below.
Below is the fourth line.
Fourth line is this only.
Only fifth one is correct.
Correct and first.


Overwriting romy.txt


In [2]:

file=open("romy.txt", encoding='utf8')
read=file.read()
file.seek(0)
    
line=1
for word in read:
    if word=='\n':
        line+=1
array=[]
for i in range(line):
    array.append(file.readline())
    
array


['First line is first and second is also first.\n',
 'Second line is third and third is given below.\n',
 'Below is the fourth line.\n',
 'Fourth line is this only.\n',
 'Only fifth one is correct.\n',
 'Correct and first.']

# Remove punctuation

In [3]:
punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
for ele in read:  
    if ele in punc:  
        read = read.replace(ele, " ")  
        

read
read=read.lower()
read

'first line is first and second is also first \nsecond line is third and third is given below \nbelow is the fourth line \nfourth line is this only \nonly fifth one is correct \ncorrect and first '

# Remove stopwords

In [4]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize


text_tokens = word_tokenize(read)

tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]

print(tokens_without_sw)

[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>
['first', 'line', 'first', 'second', 'first', 'second', 'line', 'third', 'third', 'given', 'fourth', 'line', 'fourth', 'line', 'fifth', 'correct', 'correct', 'first']


In [5]:
len(tokens_without_sw)

18

# Remove digits 

In [6]:
tokens=[]
for i in range(len(tokens_without_sw)):
               if tokens_without_sw[i].isdigit()==False:
                   tokens.append(tokens_without_sw[i])
print(len(tokens))
print(tokens)

                   

18
['first', 'line', 'first', 'second', 'first', 'second', 'line', 'third', 'third', 'given', 'fourth', 'line', 'fourth', 'line', 'fifth', 'correct', 'correct', 'first']


# Stemming

In [7]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()
for i in range(len(tokens)):
    tokens[i]=ps.stem(tokens[i])
    
print(tokens)

['first', 'line', 'first', 'second', 'first', 'second', 'line', 'third', 'third', 'given', 'fourth', 'line', 'fourth', 'line', 'fifth', 'correct', 'correct', 'first']


# Create set to avoid multiple entries

In [8]:
newset=set(tokens)
newset

{'correct', 'fifth', 'first', 'fourth', 'given', 'line', 'second', 'third'}

# Create inverted index

In [9]:
dt={}

for i in range(line):
    check=array[i].lower()
    for item in newset:
        
        if item in check:
            if item not in dt:
                dt[item]=[]
                
            if item in dt:
                dt[item].append(i+1)
                
dt



{'second': [1, 2],
 'line': [1, 2, 3, 4],
 'first': [1, 6],
 'third': [2],
 'given': [2],
 'fourth': [3, 4],
 'correct': [5, 6],
 'fifth': [5]}

# Shunting Yard Algorithm

In [10]:
def setor(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1|myset2
    return myset3

def setand(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1&myset2
    return myset3

def precedence(op): 
      
    if op == '&' or op == '|': 
        return 2
    if op == '!': 
        return 1
    return 0

def applyOp(a,b,op):
    if op=='&':
        return setand(a,b)
    if op=='|':
        return setor(a,b)
    if op=='!':
        return setnot(a,b)
    
def setnot(a,b):
    myset1=set(a)
    myset2=set(b)
    myset3= myset1-myset2
    return myset3
    
    
operator=['|', '&','(',')','!']
def evaluate(tokens): 
      
    values = []     
    ops = [] 
    i = 0
      
    while i < len(tokens):           
       
        if tokens[i] == ' ': 
            i += 1
            continue          
      
        elif tokens[i] == '(': 
            ops.append(tokens[i]) 
            
        elif tokens[i] not in operator : 
                      
            values.append(dt[tokens[i]]) 
        elif tokens[i] == ')': 
          
            while len(ops) != 0 and ops[-1] != '(': 
              
                val2 = values.pop() 
                val1 = values.pop() 
                op = ops.pop() 
                  
                values.append(applyOp(val1, val2, op)) 
              
            
            ops.pop() 
            
            
        else: 
          
           
            while (len(ops) != 0 and
                precedence(ops[-1]) >= precedence(tokens[i])): 
                          
                val2 = values.pop() 
                val1 = values.pop() 
                op = ops.pop() 
                  
                values.append(applyOp(val1, val2, op)) 
              
           
            ops.append(tokens[i]) 
          
        i += 1
    while len(ops) != 0: 
          
        val2 = values.pop() 
        val1 = values.pop() 
        op = ops.pop() 
                  
        values.append(applyOp(val1, val2, op)) 
      
    
    return values[0] 
          
    